In [917]:
import numpy as np
from plind.contour_dict import realcontour_1D, realcontour_nd
from plind.integrate import conintegrate
from plind import plmodel
from plind.contour import contour
import matplotlib.pyplot as plt
from math import factorial as fact
import itertools
from time import time

In [904]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [891]:
def cantor_pairing(a, b):
    return (0.5*(a+b)*(a+b+1)+b).astype(int)

def refine(simplices, points, delta, ndim):
    # construct edges from simplices and compute lengths
    edges = np.array([list(itertools.combinations(simp, 2)) for simp in simplices])
    diff = points[edges[:,:,0]] - points[edges[:,:,1]]
    if ndim==1:
        lengths = np.abs(np.sqrt(diff**2))
    else:
        lengths = np.abs(np.sqrt(np.sum([diff[:,:,i]**2 for i in np.arange(diff.shape[2])], axis=0)))

    # find edges that exceed delta
    where = np.where(lengths>delta)
    if len(where[0])>0:
        all_bad_edges = edges[where]
        all_edge_key = cantor_pairing(all_bad_edges[:,0], all_bad_edges[:,1]) # unique indetifier for each edge

        # make it so there is only one edge per simplex, and remove secondary edges as flagged edges
        bad_simp_ind, prim_edge_ind = np.unique(where[0], return_index=True)
        dupl_edge_key = np.delete(all_edge_key, prim_edge_ind)
        dupl_edge_ind = np.where(np.in1d(all_edge_key, dupl_edge_key))[0]

        bad_simp_ind = np.delete(where[0], dupl_edge_ind)
        bad_simps = simplices[bad_simp_ind]
        bad_edges = np.delete(all_bad_edges, dupl_edge_ind, axis=0)
        edge_key = np.delete(all_edge_key, dupl_edge_ind)

        # create new points to add to simplices and also determine the index of these new points once added to self.points
        uni_edge_key, unique_inds = np.unique(edge_key, return_index=True) # identify unique edges (as edges may be shared by simplices)
        uni_bad_edges = bad_edges[unique_inds]

        new_pnts = (points[uni_bad_edges[:, 0]] + points[uni_bad_edges[:, 1]])/2
        newpnts_inds = np.array([np.where(uni_edge_key==i)[0][0] for i in edge_key]) + len(points)

        A = np.hstack([newpnts_inds[:, None], np.array([simp[np.where(simp != e0)] for simp, e0 in zip(bad_simps, bad_edges[:,0])])])
        B = np.hstack([newpnts_inds[:, None], np.array([simp[np.where(simp != e0)] for simp, e0 in zip(bad_simps, bad_edges[:,1])])])
        new_simps = np.concatenate([A,B])

        new_simps = np.concatenate([np.delete(simplices, bad_simp_ind, axis=0), new_simps])
        new_pnts = np.concatenate([points, new_pnts])
        
        return new_simps, new_pnts
    else:
        return simplices, points



In [900]:
# 2D check
delta = 0.72
points = np.array([[0., 0.],
           [0., 1.],
           [1., 1.],
           [0.6,0.],
           [0.7, 0.7],
           [-0.001, 0.5]])
simplices = np.array([[0,1,4], [1,2,4],[2,4,3],[0,4,3],[0,1,5]])

new_simps, new_pnts = refine(simplices, points, delta, 2)
print(new_simps, '\n', new_pnts)

[[0 4 3]
 [6 1 4]
 [7 2 4]
 [8 4 3]
 [6 1 5]
 [6 0 4]
 [7 1 4]
 [8 2 4]
 [6 0 5]] 
 [[ 0.     0.   ]
 [ 0.     1.   ]
 [ 1.     1.   ]
 [ 0.6    0.   ]
 [ 0.7    0.7  ]
 [-0.001  0.5  ]
 [ 0.     0.5  ]
 [ 0.5    1.   ]
 [ 0.8    0.5  ]]


In [901]:
# 1D test
delta = 1.
points = np.array([0., 1.+1j, 2.+2j, 3.+3j, 3.1+3.1j])
simplices = np.array([[0,1],[1,2],[2,3],[3,4]])
ndim = 1

new_simps, new_pnts = refine(simplices, points, delta, 1)
print(new_simps, '\n', new_pnts)

[[3 4]
 [5 1]
 [6 2]
 [7 3]
 [5 0]
 [6 1]
 [7 2]] 
 [0. +0.j  1. +1.j  2. +2.j  3. +3.j  3.1+3.1j 0.5+0.5j 1.5+1.5j 2.5+2.5j]


In [902]:
# 3D test
delta = 1.
points = np.array([[0., 0., 0.],
                  [1., 0., 0.],
                  [0., 1., 0.],
                  [0., 0., 1.],
                  [1., 1., 1.]])
simplices = np.array([[0,1,2,3],[1,2,3,4]])

new_simps, new_pnts = refine(simplices, points, delta, 3)
print(new_simps, '\n', new_pnts)

[[5 0 2 3]
 [5 2 3 4]
 [5 0 1 3]
 [5 1 3 4]] 
 [[0.  0.  0. ]
 [1.  0.  0. ]
 [0.  1.  0. ]
 [0.  0.  1. ]
 [1.  1.  1. ]
 [0.5 0.5 0. ]]


In [935]:
# big test
Nit = 3
contour = realcontour_nd(100, (-0.5,0.5,-0.5,0.5))

t0 = time()
for i in np.arange(Nit):
    new_simps, new_pnts = refine(contour.simplices, contour.points, 0.00001, contour.ndim)
t1 = time()
print((t1-t0)/Nit)

0.5491399765014648


In [936]:
init_time = 0
refine_time = 0
for i in np.arange(Nit):
    contour = realcontour_nd(100, (-0.5,0.5,-0.5,0.5))
    t0 = time()
    contour.refine_edges(0.00001)
    t1 = time()
    refine_time += t1-t0

print((refine_time)/Nit)

    

50.4124858379364


In [912]:
len(new_simps)

1292

# Attempt 1

In [609]:
def cantor_pairing(a, b):
    return (0.5*(a+b)*(a+b+1)+b).astype(int)

def refine(simplices, points, delta, ndim):
    # construct edges from simplices and compute lengths
    edges = np.array([list(itertools.combinations(simp, 2)) for simp in simplices])
    diff = points[edges[:,:,0]] - points[edges[:,:,1]]
    if ndim==1:
        lengths = np.abs(np.sqrt(diff**2))
    else:
        lengths = np.abs(np.sqrt(np.sum([diff[:,:,i]**2 for i in np.arange(diff.shape[2])], axis=0)))
    
    # find edges that exceed delta
    where = np.where(lengths>delta)
    bad_simp_ind = np.unique(where[0])
    bad_edges = edges[where]

    # use cantor pairing function to assign unique identifier for an edge to identify duplicates
    edge_key = cantor_pairing(bad_edges[:,0], bad_edges[:,1])
    uni_edge_key, unique_inds = np.unique(edge_key, return_index=True)
    uni_bad_edges = bad_edges[unique_inds]
    
    # create new points to add to simplices and also determine the index of these new points once added to self.points
    uni_edgepts = (points[uni_bad_edges[:, 0]] + points[uni_bad_edges[:, 1]])/2
    centres = (np.sum(points[simplices[bad_simp_ind]], axis=1))/simplices.shape[1]
    new_pnts = np.concatenate([uni_edgepts, centres])
    edgepts_inds = np.array([np.where(uni_edge_key==i)[0][0] for i in edge_key]) + len(points)
    centrepts_inds = np.arange(len(centres)) + len(points) + len(uni_edgepts)

    # generate new simplices 
    new_simplices = simplices 
    for i, simp in enumerate(simplices):
        cp = centrepts_inds[i]
        edpt = edgepts_inds[where[0]==i]
        bad_e = bad_edges[where[0]==i]

        orig_e = np.array(list(itertools.combinations(simp, 2)))
        good_e = orig_e[np.invert((orig_e[:,None] == bad_e).all(2).any(1))]

        not_centre_es = np.array([list(itertools.combinations(np.concatenate([bad_e[i], [ep]]), 2)) for i, ep in enumerate(edpt)])
        not_centre_es = not_centre_es.reshape(not_centre_es.shape[0]*not_centre_es.shape[1], 2)

        A = np.concatenate([not_centre_es, orig_e])
        not_centre_es = A[np.invert((A[:, None] == orig_e).all(2).any(1))]
        not_centre_es = np.concatenate([not_centre_es, good_e])

        cstack = np.array([[cp]]*len(not_centre_es))
        new_simps = np.hstack([not_centre_es, cstack])
        new_simplices = np.concatenate([new_simplices, new_simps])

    new_simplices = np.delete(new_simplices, bad_simp_ind, axis=0)
    new_points = np.concatenate([points, new_pnts])
    return new_simplices, new_points


In [606]:
# 2D test
delta = 2.5
points = np.array([[0., -1.],
           [0., 1.],
           [3., 0.],
           [2.,2.]])
simplices = np.array([[0,1,2], [1,2,3]])

new_simps, new_pnts = refine(simplices, points, delta, 2)

print(new_pnts)
print(new_simps)

[[ 0.         -1.        ]
 [ 0.          1.        ]
 [ 3.          0.        ]
 [ 2.          2.        ]
 [ 1.5        -0.5       ]
 [ 1.5         0.5       ]
 [ 1.          0.        ]
 [ 1.66666667  1.        ]]
[[0 4 6]
 [2 4 6]
 [1 5 6]
 [2 5 6]
 [0 1 6]
 [1 5 7]
 [2 5 7]
 [1 3 7]
 [2 3 7]]


In [612]:
# 1D test
delta = 1.
points = np.array([0., 1.+1j, 2.+2j, 3.+3j, 3.1+3.1j])
simplices = np.array([[0,1],[1,2],[2,3],[3,4]])
ndim = 1

#new_simps, new_pnts = refine(simplices, points, delta, 1)

#print(new_pnts)
#print(new_simps)

 # construct edges from simplices and compute lengths
edges = np.array([list(itertools.combinations(simp, 2)) for simp in simplices])
diff = points[edges[:,:,0]] - points[edges[:,:,1]]
if ndim==1:
    lengths = np.abs(np.sqrt(diff**2))
else:
    lengths = np.abs(np.sqrt(np.sum([diff[:,:,i]**2 for i in np.arange(diff.shape[2])], axis=0)))

# find edges that exceed delta
where = np.where(lengths>delta)
bad_simp_ind = np.unique(where[0])
bad_edges = edges[where]

# use cantor pairing function to assign unique identifier for an edge to identify duplicates
edge_key = cantor_pairing(bad_edges[:,0], bad_edges[:,1])
uni_edge_key, unique_inds = np.unique(edge_key, return_index=True)
uni_bad_edges = bad_edges[unique_inds]

# create new points to add to simplices and also determine the index of these new points once added to self.points
uni_edgepts = (points[uni_bad_edges[:, 0]] + points[uni_bad_edges[:, 1]])/2
centres = (np.sum(points[simplices[bad_simp_ind]], axis=1))/simplices.shape[1]
new_pnts = np.concatenate([uni_edgepts, centres])
edgepts_inds = np.array([np.where(uni_edge_key==i)[0][0] for i in edge_key]) + len(points)
centrepts_inds = np.arange(len(centres)) + len(points) + len(uni_edgepts)

# generate new simplices 
new_simplices = simplices 
for i, simp in enumerate(simplices):
    cp = centrepts_inds[i]
    edpt = edgepts_inds[where[0]==i]
    bad_e = bad_edges[where[0]==i]

    orig_e = np.array(list(itertools.combinations(simp, 2)))
    good_e = orig_e[np.invert((orig_e[:,None] == bad_e).all(2).any(1))]

    not_centre_es = np.array([list(itertools.combinations(np.concatenate([bad_e[i], [ep]]), 2)) for i, ep in enumerate(edpt)])
    not_centre_es = not_centre_es.reshape(not_centre_es.shape[0]*not_centre_es.shape[1], 2)

    A = np.concatenate([not_centre_es, orig_e])
    not_centre_es = A[np.invert((A[:, None] == orig_e).all(2).any(1))]
    not_centre_es = np.concatenate([not_centre_es, good_e])

    cstack = np.array([[cp]]*len(not_centre_es))
    new_simps = np.hstack([not_centre_es, cstack])
    new_simplices = np.concatenate([new_simplices, new_simps])

new_simplices = np.delete(new_simplices, bad_simp_ind, axis=0)
new_points = np.concatenate([points, new_pnts])

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 3

In [619]:
orig_e

array([[0, 1]])

# developing

In [539]:
delta = 2
c = contour()
c.points = np.array([[0., -1.],
           [0., 1.],
           [3., 0.],
           [2.,2.]])
c.simplices = np.array([[0,1,2], [1,2,3]])
ndim = 2

In [540]:
max_edge_arg = np.argmax(ei_dot_ej.reshape(-1, ei_dot_ej.shape[-1]), axis=0)
print(max_edge_arg)

[2 1]


In [541]:
A = np.stack((max_edge_arg//(ndim+1), max_edge_arg%(ndim+1)), axis=1)[bad_simp_ind]
print(A)

[[0 2]
 [0 1]]


In [542]:
bad_edges = np.array([bad_simp[i][a] for i,a in enumerate(A)])
print(bad_edges)

[[0 2]
 [1 2]]


In [543]:
bad_simps = c.simplices[bad_simp_ind]

In [544]:
all(x in bad_simp[0] for x in bad_edges[0])

True

In [545]:
np.isin(c.simplices, bad_edges)

array([[ True,  True,  True],
       [ True,  True, False]])

In [546]:
bad_simp

array([[0, 1, 2],
       [1, 2, 3]])

In [547]:
np.array(list(itertools.product(c.simplices, c.simplices)))

array([[[0, 1, 2],
        [0, 1, 2]],

       [[0, 1, 2],
        [1, 2, 3]],

       [[1, 2, 3],
        [0, 1, 2]],

       [[1, 2, 3],
        [1, 2, 3]]])

In [548]:
np.subtract(c.simplices[:, None], -c.simplices[None, :])/2

array([[[0. , 1. , 2. ],
        [0.5, 1.5, 2.5]],

       [[0.5, 1.5, 2.5],
        [1. , 2. , 3. ]]])

In [549]:
c.simplices[None, :]

array([[[0, 1, 2],
        [1, 2, 3]]])

In [550]:
ei_dot_ej

array([[[ 0.,  0.],
        [ 4., 10.],
        [10.,  5.]],

       [[ 4., 10.],
        [ 0.,  0.],
        [10.,  5.]],

       [[10.,  5.],
        [10.,  5.],
        [ 0.,  0.]]])

In [551]:
np.where((ei_dot_ej>5).flatten())

(array([ 3,  4,  7, 10, 12, 14]),)

In [552]:
np.sum(ei_dot_ej>delta**2, axis=1)

array([[1, 2],
       [1, 2],
       [2, 2]])

In [553]:
np.sum(np.sum(ei_dot_ej>delta**2, axis=1), axis=0)

array([4, 6])

In [554]:
bad_simp_ind = np.where(np.sum(np.sum(ei_dot_ej>delta**2, axis=1), axis=0))[0]

In [555]:
bad_simp_ind

array([0, 1])

In [556]:
edges = np.array([list(itertools.combinations(simp, 2)) for simp in c.simplices])
print(edges)

[[[0 1]
  [0 2]
  [1 2]]

 [[1 2]
  [1 3]
  [2 3]]]


In [565]:
diff = c.points[edges[:,:,0]]-c.points[edges[:,:,1]]
lengths = np.abs(np.sqrt(np.sum([diff[:,:,i]**2 for i in np.arange(diff.shape[2])], axis=0)))
print(diff)
print(lengths)

[[[ 0. -2.]
  [-3. -1.]
  [-3.  1.]]

 [[-3.  1.]
  [-2. -1.]
  [ 1. -2.]]]
[[2.         3.16227766 3.16227766]
 [3.16227766 2.23606798 2.23606798]]


In [472]:
where = np.where(lengths>2.5)
bad_simp_ind = np.unique(where[0])
print(where)
print(bad_simp_ind)

(array([0, 0, 1]), array([1, 2, 0]))
[0 1]


In [473]:
bad_edges = edges[where]
print(bad_edges)

[[0 2]
 [1 2]
 [1 2]]


In [474]:
def cantor_pairing(a, b):
   return (0.5*(a+b)*(a+b+1)+b).astype(int)

edge_key = cantor_pairing(bad_edges[:,0], bad_edges[:,1])
print(edge_key)

[5 8 8]


In [475]:
uni_edge_key, unique_inds = np.unique(edge_key, return_index=True)
uni_bad_edges = bad_edges[unique_inds]
print(unique_inds)
print(uni_bad_edges)

[0 1]
[[0 2]
 [1 2]]


In [476]:
uni_edgepts = (c.points[uni_bad_edges[:, 0]] + c.points[uni_bad_edges[:, 1]])/2
centres = (np.sum(c.points[c.simplices[bad_simp_ind]], axis=1))/c.simplices.shape[1]
new_pnts = np.concatenate([uni_edgepts, centres])
print(edgepts)
print(centres)

[[ 1.5 -0.5]
 [ 1.5  0.5]]
[[1.         0.        ]
 [1.66666667 1.        ]]


In [477]:
edgepts_inds = np.array([np.where(uni_edge_key==i)[0][0] for i in edge_key]) + len(c.points)
centrepts_inds = np.arange(len(centres)) + len(c.points) + len(uni_edgepts)
print(edgepts_inds)
print(centrepts_inds)

[4 5 5]
[6 7]


In [478]:
print(where)
print(c.simplices)
print(edgepts_inds)
print(centrepts_inds)

(array([0, 0, 1]), array([1, 2, 0]))
[[0 1 2]
 [1 2 3]]
[4 5 5]
[6 7]


In [491]:
new_simplices = c.simplices
for i, simp in enumerate(c.simplices):
    cp = centrepts_inds[i]
    edpt = edgepts_inds[where[0]==i]
    bad_e = bad_edges[where[0]==i]

    orig_e = np.array(list(itertools.combinations(simp, 2)))
    good_e = orig_e[np.invert((orig_e[:,None] == bad_e).all(2).any(1))]

    not_centre_es = np.array([list(itertools.combinations(np.concatenate([bad_e[i], [ep]]), 2)) for i, ep in enumerate(edpt)])
    not_centre_es = not_centre_es.reshape(not_centre_es.shape[0]*not_centre_es.shape[1], 2)

    A = np.concatenate([not_centre_es, orig_e])
    not_centre_es = A[np.invert((A[:, None] == orig_e).all(2).any(1))]
    not_centre_es = np.concatenate([not_centre_es, good_e])

    cstack = np.array([[cp]]*len(not_centre_es))
    new_simps = np.hstack([not_centre_es, cstack])
    new_simplices = np.concatenate([new_simplices, new_simps])

new_simplices = np.delete(new_simplices, bad_simp_ind, axis=0)
new_points = np.concatenate([c.points, new_pnts])
print(new_simplices)
print(new_points)


[[0 4 6]
 [2 4 6]
 [1 5 6]
 [2 5 6]
 [0 1 6]
 [1 5 7]
 [2 5 7]
 [1 3 7]
 [2 3 7]]
[[ 0.         -1.        ]
 [ 0.          1.        ]
 [ 3.          0.        ]
 [ 2.          2.        ]
 [ 1.5        -0.5       ]
 [ 1.5         0.5       ]
 [ 1.          0.        ]
 [ 1.66666667  1.        ]]


array([[0, 4, 6],
       [2, 4, 6],
       [1, 5, 6],
       [2, 5, 6],
       [0, 1, 6],
       [1, 5, 7],
       [2, 5, 7],
       [1, 3, 7],
       [2, 3, 7]])

In [447]:
new_simplices

array([array([[0, 4, 6],
       [2, 4, 6],
       [1, 5, 6],
       [2, 5, 6],
       [0, 1, 6]]),
       array([[1, 5, 7],
       [2, 5, 7],
       [1, 3, 7],
       [2, 3, 7]])], dtype=object)

In [443]:
cp = centrepts_inds[0]
edpt = edgepts_inds[where[0]==0]
bad_e = bad_edges[where[0]==0]

orig_e = np.array(list(itertools.combinations(c.simplices[0], 2)))
good_e = orig_e[np.invert((orig_e[:,None] == bad_e).all(2).any(1))]

not_centre_es = np.array([list(itertools.combinations(np.concatenate([bad_e[i], [ep]]), 2)) for i, ep in enumerate(edpt)])
not_centre_es = not_centre_es.reshape(not_centre_es.shape[0]*not_centre_es.shape[1], 2)

A = np.concatenate([not_centre_es, orig_e])
not_centre_es = A[np.invert((A[:, None] == orig_e).all(2).any(1))]
not_centre_es = np.concatenate([not_centre_es, good_e])

cstack = np.array([[cp]]*len(not_centre_es))
new_simps = np.hstack([not_centre_es, cstack])

print(cp)
print(edpt)
print(bad_e)
print(orig_e)
print(good_e)
print(not_centre_es)
print(new_simps)


6
[4 5]
[[0 2]
 [1 2]]
[[0 1]
 [0 2]
 [1 2]]
[[0 1]]
[[0 4]
 [2 4]
 [1 5]
 [2 5]
 [0 1]]
[[0 4 6]
 [2 4 6]
 [1 5 6]
 [2 5 6]
 [0 1 6]]


In [438]:
np.hstack([not_centre_es, cstack])


array([[6, 4, 6],
       [6, 4, 6],
       [6, 5, 6],
       [6, 5, 6],
       [6, 1, 6]])

In [435]:
not_centre_es

array([[6, 4],
       [6, 4],
       [6, 5],
       [6, 5],
       [6, 1]])

In [409]:
not_centre_es

array([[[0, 2],
        [0, 4],
        [2, 4]],

       [[1, 2],
        [1, 5],
        [2, 5]]])

In [647]:
ndim = 3
simp = np.array([0,1,2,3])
bad_e = np.array([[0,1]])
edpt = np.array([4])
cp = 5

orig_face = np.array(list(itertools.combinations(simp, ndim)))
bad_face_loc = (np.sum(np.isin(orig_face, bad_e), axis=1) == 2)
bad_face = orig_face[bad_face_loc]
good_face = orig_face[np.invert(bad_face_loc)]

not_centre_faces = np.array([list(itertools.combinations(np.concatenate([bad_face[i], [ep]]), ndim)) for i, ep in enumerate(edpt)])
# not_centre_es = not_centre_es.reshape(not_centre_es.shape[0]*not_centre_es.shape[1], 2)

# A = np.concatenate([not_centre_es, orig_e])
# not_centre_es = A[np.invert((A[:, None] == orig_e).all(2).any(1))]
# not_centre_es = np.concatenate([not_centre_es, good_e])

# cstack = np.array([[cp]]*len(not_centre_es))
# new_simps = np.hstack([not_centre_es, cstack])
# new_simplices = np.concatenate([new_simplices, new_simps])

In [642]:
orig_face

array([[0, 1, 2],
       [0, 1, 3],
       [0, 2, 3],
       [1, 2, 3]])

In [643]:
good_face

array([[0, 2, 3],
       [1, 2, 3]])

In [644]:
bad_face

array([[0, 1, 2],
       [0, 1, 3]])

In [648]:
not_centre_faces

array([[[0, 1, 2],
        [0, 1, 4],
        [0, 2, 4],
        [1, 2, 4]]])

# Another attempt

In [884]:
delta = 0.72
points = np.array([[0., 0.],
           [0., 1.],
           [1., 1.],
           [0.6,0.],
           [0.7, 0.7],
           [-0.001, 0.5]])
simplices = np.array([[0,1,4], [1,2,4],[2,4,3],[0,4,3],[0,1,5]])

In [885]:
# construct edges from simplices and compute lengths
edges = np.array([list(itertools.combinations(simp, 2)) for simp in simplices])
diff = points[edges[:,:,0]] - points[edges[:,:,1]]
if ndim==1:
    lengths = np.abs(np.sqrt(diff**2))
else:
    lengths = np.abs(np.sqrt(np.sum([diff[:,:,i]**2 for i in np.arange(diff.shape[2])], axis=0)))

# find edges that exceed delta
where = np.where(lengths>delta)
if len(where[0])>0:
    all_bad_edges = edges[where]
    all_edge_key = cantor_pairing(all_bad_edges[:,0], all_bad_edges[:,1]) # unique indetifier for each edge

    # make it so there is only one edge per simplex, and remove secondary edges as flagged edges
    bad_simp_ind, prim_edge_ind = np.unique(where[0], return_index=True)
    dupl_edge_key = np.delete(all_edge_key, prim_edge_ind)
    dupl_edge_ind = np.where(np.in1d(all_edge_key, dupl_edge_key))[0]

    bad_simp_ind = np.delete(where[0], dupl_edge_ind)
    bad_simps = simplices[bad_simp_ind]
    bad_edges = np.delete(all_bad_edges, dupl_edge_ind, axis=0)
    edge_key = np.delete(all_edge_key, dupl_edge_ind)

    # create new points to add to simplices and also determine the index of these new points once added to self.points
    uni_edge_key, unique_inds = np.unique(edge_key, return_index=True) # identify unique edges (as edges may be shared by simplices)
    uni_bad_edges = bad_edges[unique_inds]

    new_pnts = (points[uni_bad_edges[:, 0]] + points[uni_bad_edges[:, 1]])/2
    newpnts_inds = np.array([np.where(uni_edge_key==i)[0][0] for i in edge_key]) + len(points)
    
    A = np.hstack([newpnts_inds[:, None], np.array([simp[np.where(simp != e0)] for simp, e0 in zip(bad_simps, bad_edges[:,0])])])
    B = np.hstack([newpnts_inds[:, None], np.array([simp[np.where(simp != e0)] for simp, e0 in zip(bad_simps, bad_edges[:,1])])])
    new_simps = np.concatenate([A,B])
    
    new_simps = np.concatenate([np.delete(simplices, bad_simp_ind, axis=0), new_simps])
    new_pnts = np.concatenate([points, new_pnts])


In [828]:
lengths

array([[1.        , 0.98994949, 0.76157731],
       [1.        , 0.76157731, 0.42426407],
       [0.42426407, 1.07703296, 0.70710678],
       [0.98994949, 0.6       , 0.70710678],
       [1.        , 0.500001  , 0.500001  ]])

In [829]:
where

(array([0, 0, 0, 1, 1, 2, 3, 4]), array([0, 1, 2, 0, 1, 1, 0, 0]))

In [830]:
all_bad_edges

array([[0, 1],
       [0, 4],
       [1, 4],
       [1, 2],
       [1, 4],
       [2, 3],
       [0, 4],
       [0, 1]])

In [831]:
all_edge_key

array([ 2, 14, 19,  8, 19, 18, 14,  2])

In [832]:
prim_edge_ind

array([0, 3, 5, 6, 7])

In [833]:
dupl_edge_key

array([14, 19, 19])

In [834]:
dupl_edge_ind

array([1, 2, 4, 6])

In [835]:
bad_simps

array([[0, 1, 4],
       [1, 2, 4],
       [2, 4, 3],
       [0, 1, 5]])

In [836]:
bad_edges

array([[0, 1],
       [1, 2],
       [2, 3],
       [0, 1]])

In [837]:
edge_key

array([ 2,  8, 18,  2])

In [838]:
new_pnts

array([[0. , 0.5],
       [0.5, 1. ],
       [0.8, 0.5]])

In [839]:
newpnts_inds

array([6, 7, 8, 6])

In [841]:
bad_edges

array([[0, 1],
       [1, 2],
       [2, 3],
       [0, 1]])

In [843]:
np.hstack(newpnts_inds[:, None]

array([[6],
       [7],
       [8],
       [6]])

In [863]:
bad_edges

array([[0, 1],
       [1, 2],
       [2, 3],
       [0, 1]])

In [847]:
bad_simps

array([[0, 1, 4],
       [1, 2, 4],
       [2, 4, 3],
       [0, 1, 5]])

In [864]:
A = np.hstack([newpnts_inds[:, None], np.array([simp[np.where(simp != e0)] for simp, e0 in zip(bad_simps, bad_edges[:,0])])])
B = np.hstack([newpnts_inds[:, None], np.array([simp[np.where(simp != e0)] for simp, e0 in zip(bad_simps, bad_edges[:,1])])])
new_simps = np.concatenate([A,B])


In [865]:
new_simps

array([[6, 1, 4],
       [7, 2, 4],
       [8, 4, 3],
       [6, 1, 5],
       [6, 0, 4],
       [7, 1, 4],
       [8, 2, 4],
       [6, 0, 5]])

In [862]:
B

array([[6, 0, 4],
       [7, 1, 4],
       [8, 2, 4],
       [6, 0, 5]])

In [867]:
bad_simps

array([[0, 1, 4],
       [1, 2, 4],
       [2, 4, 3],
       [0, 1, 5]])

In [881]:
np.concatenate([np.delete(simplices, [0,1,2,3,4], axis=0), new_simps])

array([[6, 1, 4],
       [7, 2, 4],
       [8, 4, 3],
       [6, 1, 5],
       [6, 0, 4],
       [7, 1, 4],
       [8, 2, 4],
       [6, 0, 5]])

In [886]:
new_pnts

array([[ 0.   ,  0.   ],
       [ 0.   ,  1.   ],
       [ 1.   ,  1.   ],
       [ 0.6  ,  0.   ],
       [ 0.7  ,  0.7  ],
       [-0.001,  0.5  ],
       [ 0.   ,  0.5  ],
       [ 0.5  ,  1.   ],
       [ 0.8  ,  0.5  ]])

In [887]:
new_simps

array([[0, 4, 3],
       [6, 1, 4],
       [7, 2, 4],
       [8, 4, 3],
       [6, 1, 5],
       [6, 0, 4],
       [7, 1, 4],
       [8, 2, 4],
       [6, 0, 5]])

In [888]:
bad_edges

array([[0, 1],
       [1, 2],
       [2, 3],
       [0, 1]])

In [889]:
new_pnts

array([[ 0.   ,  0.   ],
       [ 0.   ,  1.   ],
       [ 1.   ,  1.   ],
       [ 0.6  ,  0.   ],
       [ 0.7  ,  0.7  ],
       [-0.001,  0.5  ],
       [ 0.   ,  0.5  ],
       [ 0.5  ,  1.   ],
       [ 0.8  ,  0.5  ]])

In [890]:
points

array([[ 0.   ,  0.   ],
       [ 0.   ,  1.   ],
       [ 1.   ,  1.   ],
       [ 0.6  ,  0.   ],
       [ 0.7  ,  0.7  ],
       [-0.001,  0.5  ]])